# TRÍCH XUẤT THÔNG TIN METADATA CỦA CÁC LOCATION theo OPENAQ

In [1]:
# CELL Lấy đầy đủ thông tin metadata

import requests
from datetime import datetime, timezone
import pandas as pd
from dotenv import load_dotenv
load_dotenv()  # loads .env in project root

import os
API_KEY = os.getenv("OPENAQ_API_KEY")
if not API_KEY:
    raise RuntimeError("OPENAQ_API_KEY not set in environment (.env missing or not loaded)")

BASE_URL = "https://api.openaq.org/v3"
headers = {"X-API-Key": API_KEY}

bbox = "105.7,20.9,106.0,21.2"
print("Đang lấy thông tin các trạm từ API OpenAQ (bao gồm cả tọa độ)...")
resp = requests.get(f"{BASE_URL}/locations?bbox={bbox}&limit=1000", headers=headers).json()
locations_in_bbox = resp.get("results", [])

stations_info = {}
earliest_overall_date = datetime.now(timezone.utc)

if not locations_in_bbox:
    print("Không tìm thấy trạm nào trong khu vực đã chọn.")
else:
    print(f"Tìm thấy {len(locations_in_bbox)} trạm. Bắt đầu xử lý thông tin...")
    for loc in locations_in_bbox:
        loc_id = loc['id']
        
        # Lấy tọa độ từ trường 'coordinates'
        coords = loc.get('coordinates', {})
        lat = coords.get('latitude')
        lon = coords.get('longitude')
        
        # **KIỂM TRA QUAN TRỌNG**: Bỏ qua trạm nếu không có tọa độ
        if lat is None or lon is None:
            print(f"  - Trạm ID {loc_id} ({loc.get('name', 'N/A')}) bị bỏ qua do thiếu tọa độ.")
            continue

        # Lấy ngày bắt đầu
        dt_first_obj = loc.get("datetimeFirst")
        if dt_first_obj and dt_first_obj.get("utc"):
            first_dt_str = dt_first_obj["utc"]
            start_date = datetime.fromisoformat(first_dt_str.replace("Z", "+00:00"))
            if start_date < earliest_overall_date:
                earliest_overall_date = start_date
        else:
            print(f"  - Trạm ID {loc_id} ({loc.get('name', 'N/A')}) bị bỏ qua do thiếu 'datetimeFirst'.")
            continue
            
        # Lấy ngày kết thúc
        dt_last_obj = loc.get("datetimeLast")
        if dt_last_obj and dt_last_obj.get("utc"):
            last_dt_str = dt_last_obj["utc"]
            end_date = datetime.fromisoformat(last_dt_str.replace("Z", "+00:00"))
        else:
            end_date = datetime.now(timezone.utc)
            
        # Lưu tất cả thông tin cần thiết vào dictionary
        stations_info[loc_id] = {
            "name": loc.get('name', 'Unknown'),
            "start_date": start_date,
            "end_date": end_date,
            "lat": lat,
            "lon": lon
        }
    
    print("-" * 20)
    print(f"Cell code này: Đã thu thập thông tin của {len(stations_info)} trạm hợp lệ (có cả tọa độ).")

Đang lấy thông tin các trạm từ API OpenAQ (bao gồm cả tọa độ)...
Tìm thấy 33 trạm. Bắt đầu xử lý thông tin...
  - Trạm ID 18 (SPARTAN - Vietnam Acad. Sci.) bị bỏ qua do thiếu 'datetimeFirst'.
  - Trạm ID 307169 (nồng độ pm) bị bỏ qua do thiếu 'datetimeFirst'.
--------------------
Cell code này: Đã thu thập thông tin của 31 trạm hợp lệ (có cả tọa độ).


In [2]:
# CELL Xuất Metadata ra file CSV

import pandas as pd
import os

if 'stations_info' not in locals() or not stations_info:
    print("Vui lòng chạy Cell 2 trước để tạo metadata.")
else:
    df_metadata = pd.DataFrame.from_dict(stations_info, orient='index')
    df_metadata = df_metadata.reset_index().rename(columns={'index': 'location_id'})
    
    metadata_filename = "stations_metadata.csv"
    df_metadata.to_csv(metadata_filename, index=False, encoding='utf-8-sig')
    
    print(f"\n---> Cell code HOÀN TẤT <---")
    print(f"Đã xuất thành công thông tin của {len(df_metadata)} trạm ra file: '{metadata_filename}'")
    print("\nXem trước 5 dòng đầu của file metadata:")
    print(df_metadata.head())


---> Cell code HOÀN TẤT <---
Đã xuất thành công thông tin của 31 trạm ra file: 'stations_metadata.csv'

Xem trước 5 dòng đầu của file metadata:
   location_id                          name                start_date  \
0         2539     US Diplomatic Post: Hanoi 2016-01-30 01:00:00+00:00   
1         7441                         Hanoi 2016-11-09 18:00:00+00:00   
2      1285357  SPARTAN - Vietnam Acad. Sci. 2019-11-28 03:00:00+00:00   
3      2161290                      An Khánh 2024-01-29 06:00:00+00:00   
4      2161291                      Cầu Diễn 2024-01-22 01:00:00+00:00   

                   end_date        lat         lon  
0 2016-11-09 16:00:00+00:00  21.021770  105.819002  
1 2025-04-09 15:00:00+00:00  21.021939  105.818806  
2 2020-06-23 14:00:00+00:00  21.047800  105.800000  
3 2025-06-10 03:00:00+00:00  21.002400  105.718100  
4 2024-12-11 14:00:00+00:00  21.039800  105.765200  
